In [2]:
from typing import List

- if length == 1 -> just **handle that single one**
- if length > 1 -> **lastIx, currIx = 0, 1**
- if consecutive (currIx == lastIx + 1) same address and amounts ![0, 0]: 1155 -> **loop until different address**
- if consecutive amounts [0, 0], then 721 -> **handle lend on last**. **lastIx = currIx, currIx++**
- if consecutive amounts [0, 0] and same address: same 721 but different tokenIds transfer -> **handle lend on last**
- if consecutive same address, and now new address -> it was a batch transfer for 1155. Handle **lend for 1155 on last. Set last to current, set current to next; continue**


In [3]:
from enum import Enum

In [4]:
class NFT(Enum):
    E721 = 1
    E1155 = 2

In [5]:
class NFTAddress:
    def __init__(self, address: str, nft: NFT):
        self.address = address
        self.nft = nft
    
    def __repr__(self):
        return f"NFTAddress(address={self.address},nft={self.nft.name})"

In [6]:
def is721(_nft: NFT):
    return _nft.value == NFT.E721.value

In [7]:
def is1155(_nft: NFT):
    return  _nft.value == NFT.E1155.value

In [8]:
def handle_lend(_nft: List[NFTAddress], _amounts: List[int]):
    if is721(_nft[0].nft):
        print(f'handle_lend 721 | {_nft} {str(_amounts)}')
    elif is1155(_nft[0].nft):
        print(f'handle_lend 1155 | {_nft} {str(_amounts)}')
    else:
        return Exception('invalid token type')

In [9]:
def handle_rent(_nft: List[NFTAddress], _amounts: List[int]):
    if is721(_nft[0].nft):
        print(f'handle_rent 721 | {_nft} {str(_amounts)}')
    elif is1155(_nft[0].nft):
        print(f'handle_rent 1155 | {_nft} {str(_amounts)}')
    else:
        return Exception('invalid token type')

In [12]:
# takes nfts as inputs
# different by (i)  address
# type         (ii) 721 or 1155
# creates groups, by type
# assumes that NFTs have been passed ordered
# i.e. [721, 1155, 1155]. Otherwise, no gas efficiencies
def bundle_call(nfts: List[NFTAddress]):
  # validation
  nfts_len = len(nfts)
  if nfts_len == 0:
    raise Exception("nothing to bundle")
  if nfts_len == 1:
    print(nfts[0])

  # grouping logic
  last_ix = 0
  curr_ix = 1

  # loop until curr_ix == nfts_len
  while curr_ix != nfts_len:
    # increment the curr_ix if:
    # - nfts[curr_ix] is 1155 and address is same as nfts[last_ix]
    if is1155(nfts[curr_ix].nft) and nfts[curr_ix].address == nfts[last_ix].address:
      curr_ix += 1
    else:
      print(f"calling action on [{last_ix}, {curr_ix})")
      last_ix = curr_ix
      curr_ix += 1

  print(f"calling action on [{last_ix}, {curr_ix})")


In [13]:
bundle_call(
  [
    NFTAddress('0x123', NFT.E721),
    NFTAddress('0x223', NFT.E1155),
    NFTAddress('0x223', NFT.E1155),
    NFTAddress('0x234', NFT.E721),
    NFTAddress('0x345', NFT.E721),
    NFTAddress('0x223', NFT.E1155),
    NFTAddress('0x223', NFT.E1155),
    NFTAddress('0x345', NFT.E1155),
  ]
)

calling action on [0, 1)
calling action on [1, 3)
calling action on [3, 4)
calling action on [4, 5)
calling action on [5, 7)
calling action on [7, 8)


In [16]:
def lend(_nft: List[NFTAddress], _amounts: List[int]):

    nft_len = len(_nft)    
    if nft_len < 2:
        handle_lend([_nft[0]], [_amounts[0]])
        return

    last_ix, curr_ix = 0, 1
    while curr_ix < nft_len:

        if _nft[last_ix].address == _nft[curr_ix].address \
            and _amounts[curr_ix] != 0:
            curr_ix += 1
            continue

        handle_lend(_nft[last_ix:curr_ix], _amounts[last_ix:curr_ix])

        last_ix = curr_ix
        curr_ix += 1

    handle_lend(_nft[last_ix:], _amounts[last_ix:])

In [22]:
lend(
    [
        NFTAddress('0x123', NFT.E721),
        NFTAddress('0x223', NFT.E1155),
        NFTAddress('0x223', NFT.E1155),
        NFTAddress('0x234', NFT.E721),
        NFTAddress('0x345', NFT.E721),
        NFTAddress('0x223', NFT.E1155),
        NFTAddress('0x223', NFT.E1155),
        NFTAddress('0x345', NFT.E1155),
    ],
    [
        0,
        1,
        2,
        0,
        0,
        3,
        4,
        10
    ]
)

handle_lend 721 | [NFTAddress(address=0x123,nft=E721)] [0]
handle_lend 1155 | [NFTAddress(address=0x223,nft=E1155), NFTAddress(address=0x223,nft=E1155)] [1, 2]
handle_lend 721 | [NFTAddress(address=0x234,nft=E721)] [0]
handle_lend 721 | [NFTAddress(address=0x345,nft=E721)] [0]
handle_lend 1155 | [NFTAddress(address=0x223,nft=E1155), NFTAddress(address=0x223,nft=E1155)] [3, 4]
handle_lend 1155 | [NFTAddress(address=0x345,nft=E1155)] [10]
